# Named Entity Recognition using pretrained BERT
### Instructor : Prof. Sudeshna Sarkar

## Name: Angana Mondal <br> 
## Roll Number: 19IE10039 

In [48]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [1]:
import numpy as np 
import pandas as pd 

import os
#print(os.listdir("../input"))
!pip install pytorch-pretrained-bert
!pip install seqeval


     |████████████████████████████████| 123 kB 4.4 MB/s 
     |████████████████████████████████| 131 kB 18.7 MB/s 
     |████████████████████████████████| 79 kB 7.1 MB/s 
     |████████████████████████████████| 8.1 MB 30.3 MB/s 
     |████████████████████████████████| 138 kB 33.7 MB/s 
     |████████████████████████████████| 127 kB 50.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 43 kB 1.8 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=e8bc5a6761f1e76aa0a72bb59f8a9f4e4c07fd78fb3b1361cf970dabe76a53fe
  Stored in directory

Reading the data

In [2]:
dframe = pd.read_csv("/content/A3_dataset.csv", encoding = "ISO-8859-1", error_bad_lines=False)

b'Skipping line 7200: expected 25 fields, saw 26\n'


In [3]:
dframe.head()

,Unnamed: 0,lemma,next-lemma,next-next-lemma,next-next-pos,next-next-shape,next-next-word,next-pos,next-shape,next-word,pos,prev-iob,prev-lemma,prev-pos,prev-prev-iob,prev-prev-lemma,prev-prev-pos,prev-prev-shape,prev-prev-word,prev-shape,prev-word,sentence_idx,shape,word,tag
0,0,thousand,of,demonstr,NNS,lowercase,demonstrators,IN,lowercase,of,NNS,__START1__,__start1__,__START1__,__START2__,__start2__,__START2__,wildcard,__START2__,wildcard,__START1__,1.0,capitalized,Thousands,O
1,1,of,demonstr,have,VBP,lowercase,have,NNS,lowercase,demonstrators,IN,O,thousand,NNS,__START1__,__start1__,__START1__,wildcard,__START1__,capitalized,Thousands,1.0,lowercase,of,O
2,2,demonstr,have,march,VBN,lowercase,marched,VBP,lowercase,have,NNS,O,of,IN,O,thousand,NNS,capitalized,Thousands,lowercase,of,1.0,lowercase,demonstrators,O
3,3,have,march,through,IN,lowercase,through,VBN,lowercase,marched,VBP,O,demonstr,NNS,O,of,IN,lowercase,of,lowercase,demonstrators,1.0,lowercase,have,O
4,4,march,through,london,NNP,capitalized,London,IN,lowercase,through,VBN,O,have,VBP,O,demonstr,NNS,lowercase,demonstrators,lowercase,have,1.0,lowercase,marched,O


In [4]:
dataset=dframe.drop(['Unnamed: 0', 'lemma', 'next-lemma', 'next-next-lemma', 'next-next-pos',
       'next-next-shape', 'next-next-word', 'next-pos', 'next-shape',
       'next-word', 'prev-iob', 'prev-lemma', 'prev-pos',
       'prev-prev-iob', 'prev-prev-lemma', 'prev-prev-pos', 'prev-prev-shape',
       'prev-prev-word', 'prev-shape', 'prev-word','shape'],axis=1)

#dataset = dframe.dropdframe.drop([])

In [5]:
dataset.head()

,pos,sentence_idx,word,tag
0,NNS,1.0,Thousands,O
1,IN,1.0,of,O
2,NNS,1.0,demonstrators,O
3,VBP,1.0,have,O
4,VBN,1.0,marched,O


Preprocessing the dataset

In [6]:
class SentenceGetter(object):
    
    def __init__(self, dataset):
        self.n_sent = 1
        self.dataset = dataset
        self.empty = False
        agg_func = lambda s: [(w,p, t) for w,p, t in zip(s["word"].values.tolist(),
                                                       s['pos'].values.tolist(),
                                                        s["tag"].values.tolist())]
        self.grouped = self.dataset.groupby("sentence_idx").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [12]:
print(getter.sentences[0]) #each sentence: list of tuples (word, pos, ner)

[('Thousands', 'NNS', 'O'), ('of', 'IN', 'O'), ('demonstrators', 'NNS', 'O'), ('have', 'VBP', 'O'), ('marched', 'VBN', 'O'), ('through', 'IN', 'O'), ('London', 'NNP', 'B-geo'), ('to', 'TO', 'O'), ('protest', 'VB', 'O'), ('the', 'DT', 'O'), ('war', 'NN', 'O'), ('in', 'IN', 'O'), ('Iraq', 'NNP', 'B-geo'), ('and', 'CC', 'O'), ('demand', 'VB', 'O'), ('the', 'DT', 'O'), ('withdrawal', 'NN', 'O'), ('of', 'IN', 'O'), ('British', 'JJ', 'B-gpe'), ('troops', 'NNS', 'O'), ('from', 'IN', 'O'), ('that', 'DT', 'O'), ('country', 'NN', 'O'), ('.', '.', 'O')]


In [15]:
sentences = [" ".join([s[0] for s in sent]) for sent in getter.sentences]
sentences[0]

'Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .'

In [16]:
print(sentences) #list of strings

['Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .', 'Families of soldiers killed in the conflict joined the protesters who carried banners with such slogans as " Bush Number One Terrorist " and " Stop the Bombings . "', 'They marched from the Houses of Parliament to a rally in Hyde Park .', 'Police put the number of marchers at 10,000 while organizers claimed it was 1,00,000 .', "The protest comes on the eve of the annual conference of Britain 's ruling Labor Party in the southern English seaside resort of Brighton .", "The party is divided over Britain 's participation in the Iraq conflict and the continued deployment of 8,500 British troops in that country .", 'The London march came ahead of anti-war protests today in other cities , including Rome , Paris , and Madrid .', "The International Atomic Energy Agency is to hold second day of talks in Vienna Wednesday on how to respond to Iran 

In [18]:
labels = [[s[2] for s in sent] for sent in getter.sentences]
print(labels[0]) #list of characters of a sentence: NER tags

['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-gpe', 'O', 'O', 'O', 'O', 'O']


In [19]:
tags_vals = list(set(dataset["tag"].values)) 
tag2idx = {t: i for i, t in enumerate(tags_vals)}

In [20]:
print(tags_vals) #list of every unique NER tag
print(tag2idx) #dictionary assigning an index to every NER tag

['I-geo', nan, 'B-art', 'I-per', 'I-art', 'B-org', 'B-tim', 'I-eve', 'O', 'B-geo', 'I-org', 'B-nat', 'B-per', 'B-gpe', 'I-gpe', 'B-eve', 'I-tim', 'I-nat']
{'I-geo': 0, nan: 1, 'B-art': 2, 'I-per': 3, 'I-art': 4, 'B-org': 5, 'B-tim': 6, 'I-eve': 7, 'O': 8, 'B-geo': 9, 'I-org': 10, 'B-nat': 11, 'B-per': 12, 'B-gpe': 13, 'I-gpe': 14, 'B-eve': 15, 'I-tim': 16, 'I-nat': 17}


Importing model pacakages

In [21]:
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam
from tqdm import tqdm, trange

Setting up seed value

In [22]:
import random

In [23]:
# setup random seed
seed =42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True

In [24]:
MAX_LEN = 40 #maximum sentence length
bs =64 #batch size

In [25]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [26]:
torch.cuda.get_device_name(0) 

'Tesla K80'

Features generation  for pre-trained languge BERT

In [27]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

100%|██████████| 231508/231508 [00:00<00:00, 317245.52B/s]


In [28]:
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print(tokenized_texts[0])

['thousands', 'of', 'demonstrators', 'have', 'marched', 'through', 'london', 'to', 'protest', 'the', 'war', 'in', 'iraq', 'and', 'demand', 'the', 'withdrawal', 'of', 'british', 'troops', 'from', 'that', 'country', '.']


In [29]:
print(tokenized_texts) # list of list of tokens of each sentence
print(tokenized_texts[0]) #list of tokens, one sentence

[['thousands', 'of', 'demonstrators', 'have', 'marched', 'through', 'london', 'to', 'protest', 'the', 'war', 'in', 'iraq', 'and', 'demand', 'the', 'withdrawal', 'of', 'british', 'troops', 'from', 'that', 'country', '.'], ['families', 'of', 'soldiers', 'killed', 'in', 'the', 'conflict', 'joined', 'the', 'protesters', 'who', 'carried', 'banners', 'with', 'such', 'slogan', '##s', 'as', '"', 'bush', 'number', 'one', 'terrorist', '"', 'and', '"', 'stop', 'the', 'bombings', '.', '"'], ['they', 'marched', 'from', 'the', 'houses', 'of', 'parliament', 'to', 'a', 'rally', 'in', 'hyde', 'park', '.'], ['police', 'put', 'the', 'number', 'of', 'marche', '##rs', 'at', '10', ',', '000', 'while', 'organizers', 'claimed', 'it', 'was', '1', ',', '00', ',', '000', '.'], ['the', 'protest', 'comes', 'on', 'the', 'eve', 'of', 'the', 'annual', 'conference', 'of', 'britain', "'", 's', 'ruling', 'labor', 'party', 'in', 'the', 'southern', 'english', 'seaside', 'resort', 'of', 'brighton', '.'], ['the', 'party', '

In [31]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post") #creates stacked input tensor, fixed length

In [33]:
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["O"], padding="post",
                     dtype="long", truncating="post") #ground truth labels

In [34]:
attention_masks = [[float(i>0) for i in ii] for ii in input_ids]

In [35]:
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags, 
                                                            random_state=2018, test_size=0.1)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)

In [36]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [37]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [40]:
#number of labels for classification
print(len(tag2idx))

18


In [38]:
model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=len(tag2idx))

100%|██████████| 407873900/407873900 [00:29<00:00, 13843088.34B/s]


In [39]:
model.cuda()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
      

In [41]:
list(model.classifier.named_parameters()) 

[('weight', Parameter containing:
  tensor([[-0.0011,  0.0095,  0.0066,  ...,  0.0105,  0.0054,  0.0139],
          [-0.0348,  0.0092,  0.0012,  ..., -0.0180,  0.0008, -0.0274],
          [ 0.0137, -0.0071, -0.0030,  ...,  0.0076, -0.0146,  0.0017],
          ...,
          [ 0.0039,  0.0199,  0.0090,  ...,  0.0376,  0.0054,  0.0063],
          [ 0.0080, -0.0017,  0.0234,  ..., -0.0142,  0.0056,  0.0135],
          [-0.0648, -0.0192,  0.0214,  ...,  0.0124,  0.0028,  0.0012]],
         device='cuda:0', requires_grad=True)), ('bias', Parameter containing:
  tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         device='cuda:0', requires_grad=True))]

In [42]:
param_optimizer = list(model.classifier.named_parameters()) 
optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

In [43]:
from seqeval.metrics import f1_score

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [54]:
epochs = 5
max_grad_norm = 1.0

for _ in trange(epochs, desc="Epoch"):
    # TRAINING loop
    model.train()

    running_loss=0 #this is the train_loss
    
    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        #zero-out the gradients
        optimizer.zero_grad()
        
        # forward pass
        output = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        current_loss=output
       
        # backward pass
        current_loss.backward()
        
        # track train loss
        running_loss=running_loss+current_loss.item()
        
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        
        # update parameters
        optimizer.step()
        
    # print train loss per epoch
    print("\nAverage training loss: ",(running_loss/len(train_dataloader)))

  
   


Epoch:  20%|██        | 1/5 [00:08<00:35,  8.87s/it]


Average training loss:  2.435664701461792


Epoch:  40%|████      | 2/5 [00:17<00:26,  8.84s/it]


Average training loss:  2.379415249824524


Epoch:  60%|██████    | 3/5 [00:26<00:17,  8.82s/it]


Average training loss:  2.3230634212493895


Epoch:  80%|████████  | 4/5 [00:35<00:08,  8.81s/it]


Average training loss:  2.2805511713027955


Epoch: 100%|██████████| 5/5 [00:44<00:00,  8.81s/it]


Average training loss:  2.2235549211502077


In [55]:
 # saving model after the final epoch

 torch.save(model,'/content/gdrive/MyDrive/NLP_Assignment_3/bert_model.pth')

In [56]:
#Loading the saved model

loaded_model = torch.load('/content/gdrive/MyDrive/NLP_Assignment_3/bert_model.pth')

In [64]:
# evaluating the saved model on the validation set and predict the value also

loaded_model.eval()
predictions, ground_truth_labels = [], []
Val_Loss=0

# Predictions on the validation set

for batch in valid_dataloader:
  batch = tuple(t.to(device) for t in batch)
  # Unpacking the inputs from dataloader
  b_input_ids, b_input_mask, b_labels = batch

  with torch.no_grad():# we do not store gradients, so that operation is fast and there is less memory usage
    # Forward pass to calculate predictions
    preds = loaded_model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

  Val_Loss = Val_Loss + preds.mean().item()

  # In order to convert CUDA:device type tensor to numpy, first we need to move the prediction tensors and labels to the CPU
  preds = preds.detach().cpu().numpy()
  b_labels = b_labels.to('cpu').numpy()
  
  # We store the predictions and ground truth labels for calculation of validation accuracy
  predictions.append(preds)
  ground_truth_labels.append(b_labels)


predictions = np.concatenate(predictions, axis=0) 
ground_truth_labels = np.concatenate(ground_truth_labels, axis=0) 
accuracy = flat_accuracy(predictions, ground_truth_labels)
print("Validation Loss: ", abs(Val_Loss))
print("Validation Accuracy: ", accuracy)

Validation Loss:  0.9703524708747864
Validation Accuracy:  0.8776515151515152


In [69]:
# Printing the predicted outputs from the validation set using the loaded model

pred_flattened = np.argmax(predictions, axis=2).flatten()
labels_flattened = ground_truth_labels.flatten()

tag_list=list(tag2idx.keys())
tag_idx_list=list(tag2idx.values())

tag_predictions=[tag_list[tag_idx_list.index(element)] for element in pred_flattened]
tag_labels=[tag_list[tag_idx_list.index(element)] for element in labels_flattened]

output_data = {'Prediction index':pred_flattened, 'Ground truth index':labels_flattened, 'Prediction tag':tag_predictions, 'Ground truth tag':tag_labels}

output_data_df=pd.DataFrame(output_data)
print(output_data_df)

      Prediction index  Ground truth index Prediction tag Ground truth tag
0                    8                   8              O                O
1                    8                   8              O                O
2                    8                   8              O                O
3                    8                   8              O                O
4                    8                   9              O            B-geo
...                ...                 ...            ...              ...
2635                 8                   8              O                O
2636                 8                   8              O                O
2637                 8                   8              O                O
2638                 8                   8              O                O
2639                 8                   8              O                O

[2640 rows x 4 columns]


In [70]:
#Saving the outputs of the model on validation set as csv file

output_data_df.to_csv('/content/gdrive/MyDrive/NLP_Assignment_3/outputs_on_validation_set.csv')